## setup

In [1]:
from unsloth.chat_templates import standardize_sharegpt, get_chat_template, train_on_responses_only
from unsloth import FastLanguageModel
from transformers import TrainingArguments, DataCollatorForSeq2Seq, AutoTokenizer, AutoModelForCausalLM, Trainer, DataCollatorForLanguageModeling, BitsAndBytesConfig 
from huggingface_hub import login
from dotenv import load_dotenv
from datasets import load_dataset
import os
import wandb
import json
from trl import SFTTrainer #, SFTConfig
from pipeline.main import run_eval
# from peft import LoraConfig, get_peft_model
# from pipeline.main import run_eval
# from IPython.display import display, HTML
# import torch as t

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [2]:
load_dotenv()
login(token = os.environ['HF_TOKEN'])
wandb.login()

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.
wandb: Currently logged in as: atharva_nihalani (atharva_nihalani-brown-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

## Test RunEval

In [3]:
model = AutoModelForCausalLM.from_pretrained('/root/srf-project/playground/llama3-java-finetune-a/checkpoint-625', device_map='auto', torch_dtype='auto')

In [5]:
model.save_pretrained('test_dir')

In [11]:
args = {
    'model': 'hf/local',
    'model_path': '/root/srf-project/playground/llama3_dir_a/',
    'device': 'auto',
    'torch_dtype': 'auto'
}

run_eval('java', args, samples=164)

Output()

Compilation failed! Idx: 27

stderr: Main.java:7: error: no suitable method found for reduce(String,(a,b)->a + b)
        return string.chars().mapToObj(c -> Character.isLowerCase(c) ? Character.toUpperCase(c) : 
Character.toLowerCase(c)).reduce("", (a, b) -> a + b);
                                                                                                                   
        ^
    method Stream.reduce(Integer,BinaryOperator<Integer>) is not applicable
      (argument mismatch; String cannot be converted to Integer)
    method Stream.<U>reduce(U,BiFunction<U,? super Integer,U>,BinaryOperator<U>) is not applicable
      (cannot infer type-variable(s) U
        (actual and formal argument lists differ in length))
  where U,T are type-variables:
    U extends Object declared in method <U>reduce(U,BiFunction<U,? super T,U>,BinaryOperator<U>)
    T extends Object declared in interface Stream
Note: Some messages have been simplified; recompile with -Xdiags:verbose to get full output
1 error

Compilation failed! Idx: 50

stderr: Main.java:4: error: class Solution is public, should be declared in a file named Solution.java
public class Solution {
       ^
Main.java:32: error: cannot find symbol
            String encode_str = s.encodeShift(str);
                                 ^
  symbol:   method encodeShift(String)
  location: variable s of type Solution
2 errors

...

stderr: Main.java:4: error: class Solution is public, should be declared in a file named Solution.java
public class Solution {
       ^
1 error

Compilation failed! Idx: 154

stderr: Main.java:32: error: duplicate class: Main
public class Main {
       ^
1 error

Compilation failed! Idx: 162

stderr: Main.java:6: error: class Solution is public, should be declared in a file named Solution.java
public class Solution {
       ^
1 error

## FT Dataset

### FT Configs

In [ ]:
configs = [
    {'r': 16,  'alpha': 16, 'dropout': 0.00, 'lr': 1.0e-4, 'decay': 0.0},   # A",
    {'r': 64,  'alpha': 32, 'dropout': 0.10, 'lr': 8.0e-5, 'decay': 0.05},  # B",
    {'r': 32,  'alpha': 32, 'dropout': 0.05, 'lr': 5.0e-5, 'decay': 0.01},  # C",
    {'r': 8, 'alpha': 64, 'dropout': 0.00, 'lr': 1.5e-4, 'decay': 0.0},     # D",
    {'r': 128, 'alpha': 32, 'dropout': 0.10, 'lr': 7.0e-5 , 'decay': 0.02}, # E",
]

### FineTuning Config #A

In [ ]:
max_seq_length = 32768 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = configs[0]['r'], 
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = configs[0]['alpha'],
    lora_dropout = configs[0]['dropout'],
    bias = "none",
    use_gradient_checkpointing = "unsloth", 
    random_state = 3407,
)

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

In [ ]:
with open("/root/srf-project/data/codenet_questions/filtered_problem_descriptions.json", "r") as f:
    problem_descriptions = json.load(f)

def get_filtered_dataset(lang):
    dataset = load_dataset('iNeil77/CodeNet', lang, split='train')
    dataset = dataset.select_columns(['p_id', 'language', 'status', 'code'])
    dataset = dataset.filter(lambda x: x['status']=='Accepted')
    dataset = dataset.filter(lambda x: x['p_id'] in problem_descriptions.keys())

    return dataset

def get_train_test(dataset, train_size=10000, test_size=500):
    shuffled = dataset.shuffle(seed=47)
    train_set = shuffled.select(range(train_size))
    test_set = shuffled.select(range(train_size, train_size + test_size))

    return train_set, test_set

def add_description(row):
    description = problem_descriptions[row['p_id']]
    row['description'] = description
    return row

def add_conversations(row):
    preface = 'Read the following problem description. Fully implement a solution in Java. Your response should only contain the code, no explanations.\n\n'
    problem_description = row['description']
    code = row['code']

    row['conversations'] = [
        {
            'role': 'user', 
            'content': preface + problem_description,
        },
        {
            'role': 'assistant', 
            'content': code,
        }
    ]

    return row

def chat_format(row):
    convos = row['conversations']
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { 'text' : texts}

In [ ]:
my_dataset = get_filtered_dataset('Java')
train_set, test_set = get_train_test(my_dataset)

train_set = train_set.map(add_description, remove_columns=['p_id', 'language', 'status'])
test_set = test_set.map(add_description, remove_columns=['p_id', 'language', 'status'])

train_set = train_set.map(add_conversations, remove_columns=['code', 'description'])
test_set = test_set.map(add_conversations, remove_columns=['code', 'description'])

train_set = train_set.map(chat_format, batched=True)
test_set = test_set.map(chat_format, batched=True)

In [ ]:
def get_save_dir():
    idx = 0
    base_path = '/root/srf-project/playground/llama3-java-finetune-'
    already_existing = True

    while already_existing:
        idx += 1
        path = base_path + str(idx)
        
        if not os.path.exists(path):
            already_existing = False
            os.mkdir(path)
            return path
        
output_path = get_save_dir()

training_args = TrainingArguments(
    per_device_train_batch_size = 8,
    gradient_accumulation_steps = 2,
    warmup_ratio = 0.05,
    num_train_epochs = 1,
    eval_strategy = 'steps',
    eval_steps = 0.125,
    learning_rate = configs[0]['lr'],
    bf16 = True,
    logging_steps = 5,
    logging_first_step=True,
    optim = "adamw_8bit",
    weight_decay = configs[0]['decay'],
    lr_scheduler_type = "linear",
    seed = 3407,
    save_steps = 0.2,
    save_total_limit = 3,
    output_dir = output_path,
    report_to = "wandb", 
    run_name = 'quantized'
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_set,
    eval_dataset = test_set,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 8,
    args = training_args,
    # packing = False, 
)

In [ ]:
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

In [ ]:
trainer_stats_a = trainer.train()
wandb.finish()
del model, tokenizer, train_set, test_set, trainer

### FineTuning Config #B

In [ ]:
max_seq_length = 32768 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = configs[1]['r'], 
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = configs[1]['alpha'],
    lora_dropout = configs[1]['dropout'],
    bias = "none",
    use_gradient_checkpointing = "unsloth", 
    random_state = 3407,
)

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

In [ ]:
with open("/root/srf-project/data/codenet_questions/filtered_problem_descriptions.json", "r") as f:
    problem_descriptions = json.load(f)

def get_filtered_dataset(lang):
    dataset = load_dataset('iNeil77/CodeNet', lang, split='train')
    dataset = dataset.select_columns(['p_id', 'language', 'status', 'code'])
    dataset = dataset.filter(lambda x: x['status']=='Accepted')
    dataset = dataset.filter(lambda x: x['p_id'] in problem_descriptions.keys())

    return dataset

def get_train_test(dataset, train_size=3000, test_size=200):
    shuffled = dataset.shuffle(seed=47)
    train_set = shuffled.select(range(train_size))
    test_set = shuffled.select(range(train_size, train_size + test_size))

    return train_set, test_set

def add_description(row):
    description = problem_descriptions[row['p_id']]
    row['description'] = description
    return row

def add_conversations(row):
    preface = 'Read the following problem description. Fully implement a solution in Java. Your response should only contain the code, no explanations.\n\n'
    problem_description = row['description']
    code = row['code']

    row['conversations'] = [
        {
            'role': 'user', 
            'content': preface + problem_description,
        },
        {
            'role': 'assistant', 
            'content': code,
        }
    ]

    return row

def chat_format(row):
    convos = row['conversations']
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { 'text' : texts}

In [ ]:
my_dataset = get_filtered_dataset('Java')
train_set, test_set = get_train_test(my_dataset)

train_set = train_set.map(add_description, remove_columns=['p_id', 'language', 'status'])
test_set = test_set.map(add_description, remove_columns=['p_id', 'language', 'status'])

train_set = train_set.map(add_conversations, remove_columns=['code', 'description'])
test_set = test_set.map(add_conversations, remove_columns=['code', 'description'])

train_set = train_set.map(chat_format, batched=True)
test_set = test_set.map(chat_format, batched=True)

In [ ]:
def get_save_dir():
    idx = 0
    base_path = '/root/srf-project/playground/llama3-java-finetune-'
    already_existing = True

    while already_existing:
        idx += 1
        path = base_path + str(idx)
        
        if not os.path.exists(path):
            already_existing = False
            os.mkdir(path)
            return path
        
output_path = get_save_dir()

training_args = TrainingArguments(
    per_device_train_batch_size = 8,
    gradient_accumulation_steps = 2,
    warmup_ratio = 0.05,
    num_train_epochs = 1,
    eval_strategy = 'steps',
    eval_steps = 0.125,
    learning_rate = configs[1]['lr'],
    bf16 = True,
    logging_steps = 5,
    logging_first_step=True,
    optim = "adamw_8bit",
    weight_decay = configs[1]['decay'],
    lr_scheduler_type = "linear",
    seed = 3407,
    save_steps = 0.2,
    save_total_limit = 3,
    output_dir = output_path,
    report_to = "wandb", 
    run_name = 'quantized'
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_set,
    eval_dataset = test_set,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 8,
    args = training_args,
    # packing = False, 
)

In [ ]:
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

In [ ]:
trainer_stats_b = trainer.train()
wandb.finish()
del model, tokenizer, train_set, test_set, trainer

### FineTuning Config #C

In [ ]:
max_seq_length = 32768 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = configs[2]['r'], 
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = configs[2]['alpha'],
    lora_dropout = configs[2]['dropout'],
    bias = "none",
    use_gradient_checkpointing = "unsloth", 
    random_state = 3407,
)

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

In [ ]:
with open("/root/srf-project/data/codenet_questions/filtered_problem_descriptions.json", "r") as f:
    problem_descriptions = json.load(f)

def get_filtered_dataset(lang):
    dataset = load_dataset('iNeil77/CodeNet', lang, split='train')
    dataset = dataset.select_columns(['p_id', 'language', 'status', 'code'])
    dataset = dataset.filter(lambda x: x['status']=='Accepted')
    dataset = dataset.filter(lambda x: x['p_id'] in problem_descriptions.keys())

    return dataset

def get_train_test(dataset, train_size=3000, test_size=200):
    shuffled = dataset.shuffle(seed=47)
    train_set = shuffled.select(range(train_size))
    test_set = shuffled.select(range(train_size, train_size + test_size))

    return train_set, test_set

def add_description(row):
    description = problem_descriptions[row['p_id']]
    row['description'] = description
    return row

def add_conversations(row):
    preface = 'Read the following problem description. Fully implement a solution in Java. Your response should only contain the code, no explanations.\n\n'
    problem_description = row['description']
    code = row['code']

    row['conversations'] = [
        {
            'role': 'user', 
            'content': preface + problem_description,
        },
        {
            'role': 'assistant', 
            'content': code,
        }
    ]

    return row

def chat_format(row):
    convos = row['conversations']
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { 'text' : texts}

In [ ]:
my_dataset = get_filtered_dataset('Java')
train_set, test_set = get_train_test(my_dataset)

train_set = train_set.map(add_description, remove_columns=['p_id', 'language', 'status'])
test_set = test_set.map(add_description, remove_columns=['p_id', 'language', 'status'])

train_set = train_set.map(add_conversations, remove_columns=['code', 'description'])
test_set = test_set.map(add_conversations, remove_columns=['code', 'description'])

train_set = train_set.map(chat_format, batched=True)
test_set = test_set.map(chat_format, batched=True)

In [ ]:
def get_save_dir():
    idx = 0
    base_path = '/root/srf-project/playground/llama3-java-finetune-'
    already_existing = True

    while already_existing:
        idx += 1
        path = base_path + str(idx)
        
        if not os.path.exists(path):
            already_existing = False
            os.mkdir(path)
            return path
        
output_path = get_save_dir()

training_args = TrainingArguments(
    per_device_train_batch_size = 8,
    gradient_accumulation_steps = 2,
    warmup_ratio = 0.05,
    num_train_epochs = 1,
    eval_strategy = 'steps',
    eval_steps = 0.125,
    learning_rate = configs[2]['lr'],
    bf16 = True,
    logging_steps = 5,
    logging_first_step=True,
    optim = "adamw_8bit",
    weight_decay = configs[2]['decay'],
    lr_scheduler_type = "linear",
    seed = 3407,
    save_steps = 0.2,
    save_total_limit = 3,
    output_dir = output_path,
    report_to = "wandb", 
    run_name = 'quantized'
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_set,
    eval_dataset = test_set,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 8,
    args = training_args,
    # packing = False, 
)

In [ ]:
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

In [ ]:
trainer_stats_c = trainer.train()
wandb.finish()
del model, tokenizer, train_set, test_set, trainer

### Unsloth FT Working

In [ ]:
max_seq_length = 32768 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
)

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

In [ ]:
with open("/root/srf-project/data/codenet_questions/filtered_problem_descriptions.json", "r") as f:
    problem_descriptions = json.load(f)

def get_filtered_dataset(lang):
    dataset = load_dataset('iNeil77/CodeNet', lang, split='train')
    dataset = dataset.select_columns(['p_id', 'language', 'status', 'code'])
    dataset = dataset.filter(lambda x: x['status']=='Accepted')
    dataset = dataset.filter(lambda x: x['p_id'] in problem_descriptions.keys())

    return dataset

def get_train_test(dataset, train_size=10000, test_size=500):
    shuffled = dataset.shuffle(seed=47)
    train_set = shuffled.select(range(train_size))
    test_set = shuffled.select(range(train_size, train_size + test_size))

    return train_set, test_set

def add_description(row):
    description = problem_descriptions[row['p_id']]
    row['description'] = description
    return row

def add_conversations(row):
    preface = 'Read the following problem description. Fully implement a solution in Java. Your response should only contain the code, no explanations.\n\n'
    problem_description = row['description']
    code = row['code']

    row['conversations'] = [
        {
            'role': 'user', 
            'content': preface + problem_description,
        },
        {
            'role': 'assistant', 
            'content': code,
        }
    ]

    return row

def chat_format(row):
    convos = row['conversations']
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { 'text' : texts}

In [ ]:
my_dataset = get_filtered_dataset('Java')
train_set, test_set = get_train_test(my_dataset)

train_set = train_set.map(add_description, remove_columns=['p_id', 'language', 'status'])
test_set = test_set.map(add_description, remove_columns=['p_id', 'language', 'status'])

train_set = train_set.map(add_conversations, remove_columns=['code', 'description'])
test_set = test_set.map(add_conversations, remove_columns=['code', 'description'])

train_set = train_set.map(chat_format, batched=True)
test_set = test_set.map(chat_format, batched=True)

In [ ]:
def get_save_dir():
    idx = 0
    base_path = '/root/srf-project/playground/llama3-java-finetune-'
    already_existing = True

    while already_existing:
        idx += 1
        path = base_path + str(idx)
        
        if not os.path.exists(path):
            already_existing = False
            os.mkdir(path)
            return path
        
output_path = get_save_dir()

training_args = TrainingArguments(
    per_device_train_batch_size = 8,
    # auto_find_batch_size = True,
    gradient_accumulation_steps = 2,
    warmup_ratio = 0.05,
    num_train_epochs = 1,
    eval_strategy = 'steps',
    eval_steps = 0.05,
    learning_rate = 2e-4,
    bf16 = True,
    logging_steps = 5,
    logging_first_step=True,
    optim = "adamw_8bit",
    weight_decay = 0.01,
    lr_scheduler_type = "linear",
    seed = 3407,
    save_steps = 0.2,
    save_total_limit = 3,
    output_dir = output_path,
    report_to = "wandb", 
    run_name = 'quantized'
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_set,
    eval_dataset = test_set,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 8,
    args = training_args,
    # packing = False, 
)

In [ ]:
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

In [ ]:
trainer_stats = trainer.train()

In [ ]:
wandb.finish()

## Previous FT Code

### Load Model / Dataset

In [ ]:
'''
model_name = 'meta-llama/Llama-3.1-8B-Instruct'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=t.float16,
    bnb_4bit_use_double_quant=True,
    llm_int8_threshold=6.0,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, pad_side="left")
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    device_map="auto", 
    quantization_config=bnb_config,
)
tokenizer.pad_token = tokenizer.eos_token'''

# model_name = 'unsloth/Meta-Llama-3.1-8B-Instruct'
model_name = 'unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit'


tokenizer = AutoTokenizer.from_pretrained(model_name, pad_side="left")
tokenizer.pad_token = tokenizer.eos_token


model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    device_map="auto", 
    torch_dtype="auto",
)

In [ ]:
with open("/root/srf-project/data/codenet_questions/filtered_problem_descriptions.json", "r") as f:
    problem_descriptions = json.load(f)

def get_filtered_dataset(lang):
    dataset = load_dataset('iNeil77/CodeNet', lang, split='train')
    dataset = dataset.select_columns(['p_id', 'language', 'status', 'code'])
    dataset = dataset.filter(lambda x: x['status']=='Accepted')
    dataset = dataset.filter(lambda x: x['p_id'] in problem_descriptions.keys())

    return dataset

def get_train_test(dataset, train_size=10000, test_size=500):
    shuffled = dataset.shuffle(seed=47)
    train_set = shuffled.select(range(train_size))
    test_set = shuffled.select(range(train_size, train_size + test_size))

    return train_set, test_set

filtered = get_filtered_dataset('Java')
train_set, test_set = get_train_test(filtered, test_size=8)

In [ ]:
def add_description(row):
    description = problem_descriptions[row['p_id']]
    row['description'] = description
    return row

train_set = train_set.map(add_description)
test_set = test_set.map(add_description)

In [ ]:
def add_final_prompt(row):
    description = row['description'].strip()
    code = row['code']

    final_prompt = '\n'.join([description, '<answer>', code, '</answer>'])
    row['final_prompt'] = final_prompt

    return row

train_set = train_set.map(add_final_prompt)
test_set = test_set.map(add_final_prompt)

In [ ]:
def tokenize(record):
    final_prompt = record['final_prompt']
    msg = [
        {'role': 'user', 'content': final_prompt}
    ]

    tokens = tokenizer.apply_chat_template(msg, add_generation_prompt=True, return_dict=True)

    return tokens

final_train_set = train_set.map(tokenize, num_proc=32)
final_test_set = test_set.map(tokenize, num_proc=32)

final_train_set = final_train_set.select_columns(['input_ids', 'attention_mask'])
final_test_set = final_test_set.select_columns(['input_ids', 'attention_mask'])

### FineTuning

In [ ]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=64,
    target_modules=["q_proj", "v_proj"],  
    lora_dropout=0.00,
    bias="none",
    task_type="CAUSAL_LM"
)

peft_model = get_peft_model(model, lora_config)

In [ ]:
def get_save_dir():
    idx = 0
    base_path = '/root/srf-project/playground/llama3-java-finetune-'
    already_existing = True

    while already_existing:
        idx += 1
        path = base_path + str(idx)
        
        if not os.path.exists(path):
            already_existing = False
            os.mkdir(path)
            return path
        
output_path = get_save_dir()

training_args = TrainingArguments(
    output_dir=output_path,
    # eval_strategy='steps',
    # eval_steps=0.1,
    # eval_on_start=True,
    per_device_train_batch_size=1, 
    # auto_find_batch_size=True,
    gradient_accumulation_steps=1,
    # gradient_checkpointing=True,
    torch_empty_cache_steps=1,
    dataloader_num_workers=16,
    dataloader_persistent_workers=True,
    learning_rate=5e-4,
    num_train_epochs=1,  
    bf16=True,
    save_steps=0.2,
    save_total_limit=3,
    logging_steps=0.02,
    report_to="wandb",
    logging_first_step=True,
    run_name='chat_template',
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # For causal LM
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=final_train_set,
    eval_dataset=final_test_set,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

In [ ]:
wandb.finish()

### RunEval

In [ ]:
from pipeline.main import run_eval

args = {
    'model': 'hf/local',
    'model_path': '/root/srf-project/playground/llama3-java-finetune-a/checkpoint-625',
    'device': 'auto',
    'torch_dtype': 'auto'
}

run_eval('java', args, samples=3)

## GPU Deets

In [ ]:
import torch as t
import gc

free_memory, total_memory = t.cuda.mem_get_info()

# Convert bytes to GB
free_memory_gb = free_memory / (1024 * 1024 * 1024)
total_memory_gb = total_memory / (1024 * 1024 * 1024)
mem_used = t.cuda.device_memory_used() / (1024 ** 3)

print(f"Free GPU Memory: {free_memory_gb:.2f} GB")
print(f"Total GPU Memory: {total_memory_gb:.2f} GB")
print(f'Memory Used: {mem_used:.2f} GB')

In [ ]:
print(t.cuda.memory_allocated() / 1024**2, "MB allocated")
print(t.cuda.memory_reserved() / 1024**2, "MB reserved")

In [ ]:
print(t.cuda.memory_allocated() / 1024**2, "MB allocated")
print(t.cuda.memory_reserved() / 1024**2, "MB reserved")